<a href="https://colab.research.google.com/github/joeburns91/DSBA_6276_Project/blob/main/all_vs_last_statement_jb_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# DSBA 6276 : Notebook for AMEX Default Prediction

In [8]:
import os
if os.path.exists('DSBA_6276_Project'):
  print('Use local DSBA_6276_Project repo.')
else:
  print('Checking out DSBA_6276_Project repo ...')
  !git clone git@github.com:eagle034/DSBA_6276_Project.git

Checking out DSBA_6276_Project repo ...
Cloning into 'DSBA_6276_Project'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [10]:
!git clone https://github.com/eagle034/DSBA_6276_Project.git

Cloning into 'DSBA_6276_Project'...
fatal: could not read Username for 'https://github.com': No such device or address
